# Notebook to explore the DCF data

## Defining functions to allow for exploration

In [ ]:
import pandas as pd
import numpy as np
import h5py
import argparse
import unit_conversions as uc
import ipywidgets as widgets
import matplotlib.pyplot as plt

In [ ]:
def convert(func, x):
    '''Converts numpy arrays in place'''
    for i in range(0, len(x)):
        x[i] = func(x[i])


def rem_to_microsv(x):
    '''Converts rem to microsv'''
    return uc.micro(uc.rem_to_sv(x))


def microsv_to_rem(x):
    '''Converts microsv to rem'''
    return uc.mega(uc.sv_to_rem(x))


def psv_cm2_to_microsv_hour(x):
    '''Converts psv cm2 to microsv/hour per n/cm2/s'''
    return uc.mega(3600 * x)


def microsv_hour_to_psv_cm2(x):
    '''Converts microsv/hour per n/cm2/s to psv cm2'''
    return uc.micro(x / 3600)


def psv_cm2_to_rem_hour(x):
    '''Converts psv cm2 to rem/hour per n/cm2/s'''
    return uc.tera(3600 * x) * 100


def rem_hour_to_psv_cm2(x):
    '''Converts rem/hour per n/cm2/s to psv cm2'''
    return uc.pico(x / 3600) / 100


def read_hdf5(file, edit_type):
    '''Opens a hdf5 file for reading'''
    return h5py.File(file, edit_type)


def check_dcf_units(input_dcf_units, output_dcf_units, dose):
    '''Checks that the DCF units are valid and converts them into the desired output unit'''
    if input_dcf_units == 'pSv cm2':
        if output_dcf_units == 'microSv/h per n/cm2/s':
            convert(psv_cm2_to_microsv_hour, dose)
        elif output_dcf_units == 'rem/h per n/cm2/s':
            convert(psv_cm2_to_rem_hour, dose)
        else:
            raise ValueError('Invalid output unit!')
    elif input_dcf_units == 'rem/h per n/cm2/s':
        if output_dcf_units == 'microSv/h per n/cm2/s':
            convert(rem_to_microsv, dose)
        elif output_dcf_units == 'pSv cm2':
            convert(rem_hour_to_psv_cm2, dose)
        else:
            raise ValueError('Invalid output unit!')
    elif input_dcf_units == 'microSv/h per n/cm2/s':
        if output_dcf_units == 'rem/h per n/cm2/s':
            convert(microsv_to_rem, dose)
        elif output_dcf_units == 'pSv cm2':
            convert(microsv_hour_to_psv_cm2, dose)
        else:
            raise ValueError('Invalid output unit!')
    else:
        raise ValueError('Your DCF units are not accepted in this program.')


def check(path, input_dcf_units, output_dcf_units):
    '''Checks to see if the requested data exists / is valid and returns it in the requested units'''
    data = pd.DataFrame(file.get(path), columns=['E', 'DCF']).reset_index(drop=True)
    if input_dcf_units == output_dcf_units:
        pass
    else:
        dose = np.array(data['DCF'])
        check_dcf_units(input_dcf_units, output_dcf_units, dose)
        data['DCF'] = dose
    
    return data

def plot(data, xdata, ydata, energy_units, dcf_units):
    '''Plots the data (if requested)'''
    data.plot(kind='line', x=xdata, y=ydata, xlabel=('E ({})'.format(energy_units)), ylabel=('DCF ({})'.format(dcf_units)), legend=False, logx=True)

In [ ]:
class DCF_Data:
    def __init__(self, path, output_dcf_units):
        self.path = path
        self.output_dcf_units = output_dcf_units
        self.split_path = path.split('/')
        self.group = self.split_path[1]
        self.e_units = file.get('/'+self.group).attrs['energy_units']
        self.input_dcf_units = file.get('/'+self.group).attrs['dcf_units']
        self.data = check(self.path, self.input_dcf_units, self.output_dcf_units)

## Defining functions which allow user to see all datasets

In [ ]:
def descend_obj(obj,sep='\t'):
    """
    Iterate through groups in a HDF5 file and prints the groups and datasets names and datasets attributes
    """
    if type(obj) in [h5py._hl.group.Group,h5py._hl.files.File]:
        for key in obj.keys():
            print(sep,'-',key,':',obj[key])
            descend_obj(obj[key],sep=sep+'\t')
    elif type(obj)==h5py._hl.dataset.Dataset:
        for key in obj.attrs.keys():
            print(sep+'\t','-',key,':',obj.attrs[key])

def h5dump(path,group='/'):
    """
    print HDF5 file metadata

    group: you can give a specific group, defaults to the root group
    """
    with h5py.File(path,'r') as f:
         descend_obj(f[group])

In [ ]:
h5dump('DCFfile.h5', 'ICRP116')

In [ ]:
file = read_hdf5('DCFfile.h5', 'r')

# Over to you!

Choose your output DCF unit from the dropdown below.

In [ ]:
output_dcf_units = widgets.Dropdown(options=['microSv/h per n/cm2/s', 'rem/h per n/cm2/s', 'pSv cm2'], description = 'Output DCF units: ', disabled = False)
display(output_dcf_units)

Here you can input as many datasets as you like and plot them to compare. The datasets should be specified as paths of the form '/group/dataset/orientation'. 
The available groups are ESS, NCRP38, ICRP116. To see the datasets and orientations in each group, use the dump function above.
Follow the plotting examples below to plot your data.

In [ ]:
dcf1 = DCF_Data('/ICRP116/neutrons/PA', output_dcf_units.value)
dcf2 = DCF_Data('/ICRP116/neutrons/AP', output_dcf_units.value)
dcf3 = DCF_Data('/ICRP116/neutrons/LLAT', output_dcf_units.value)

In [ ]:
ax = dcf1.data.plot(kind='line', x=dcf1.data.columns[0], y=dcf1.data.columns[1], xlabel=('E ({})'.format(dcf1.e_units)), ylabel=('DCF ({})'.format(dcf1.output_dcf_units)), legend=False, logx=True)
dcf2.data.plot(kind='line', x=dcf2.data.columns[0], y=dcf2.data.columns[1], xlabel=('E ({})'.format(dcf2.e_units)), ylabel=('DCF ({})'.format(dcf2.output_dcf_units)), legend=False, logx=True, ax=ax)
dcf3.data.plot(kind='line', x=dcf3.data.columns[0], y=dcf3.data.columns[1], xlabel=('E ({})'.format(dcf3.e_units)), ylabel=('DCF ({})'.format(dcf3.output_dcf_units)), legend=False, logx=True, ax=ax)
plt.legend([dcf1.path, dcf2.path, dcf3.path])

## Select and plot your data here!

Make sure to close the hdf5 file when you're done!

In [ ]:
file.close()